In [1]:
#Load Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#Read in Data and Formatting
books = pd.read_csv('./BX-Books.csv',sep=';', error_bad_lines = False, encoding = 'latin-1', low_memory=False, warn_bad_lines = False)
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS','imageUrlM', 'imageUrlL']
users = pd.read_csv('./BX-Users.csv',sep=';', error_bad_lines = False, encoding = 'latin-1', low_memory=False, warn_bad_lines = False)
users.columns = ['userID','Location','Age']
ratings = pd.read_csv('./BX-Book-Ratings.csv',sep=';', error_bad_lines = False, encoding = 'latin-1', low_memory=False, warn_bad_lines = False)
ratings.columns = ['userID','ISBN','bookRating']

In [7]:
#Recommendations based on rating counts
rating_count = pd.DataFrame(ratings.groupby('ISBN')['bookRating'].count())                         
rating_count.sort_values('bookRating',ascending=False).head()

bookRating
ISBN                  
0971880107        2502
0316666343        1295
0385504209         883
0060928336         732
0312195516         723

In [12]:
#Look at Most Rated Books
most_rated_books = pd.DataFrame(['0971880107','0316666343','0385504209','0060928336','0312195516'], index=np.arange(5),columns=['ISBN'])
most_rated_books_summary = pd.merge(most_rated_books, books, on = 'ISBN')
most_rated_books_summary

ISBN                                        bookTitle     bookAuthor  \
0  0971880107                                      Wild Animus   Rich Shapero   
1  0316666343                        The Lovely Bones: A Novel   Alice Sebold   
2  0385504209                                The Da Vinci Code      Dan Brown   
3  0060928336  Divine Secrets of the Ya-Ya Sisterhood: A Novel  Rebecca Wells   
4  0312195516              The Red Tent (Bestselling Backlist)  Anita Diamant   

  yearOfPublication      publisher  \
0              2004        Too Far   
1              2002  Little, Brown   
2              2003      Doubleday   
3              1997      Perennial   
4              1998    Picador USA   

                                           imageUrlS  \
0  http://images.amazon.com/images/P/0971880107.0...   
1  http://images.amazon.com/images/P/0316666343.0...   
2  http://images.amazon.com/images/P/0385504209.0...   
3  http://images.amazon.com/images/P/0060928336.0...   
4  http://images.amazon.com/images/P/0312195516.0...   

                                           imageUrlM  \
0  http://images.amazon.com/images/P/0971880107.0...   
1  http://images.amazon.com/images/P/0316666343.0...   
2  http://images.amazon.com/images/P/0385504209.0...   
3  http://images.amazon.com/images/P/0060928336.0...   
4  http://images.amazon.com/images/P/0312195516.0...   

                                           imageUrlL  
0  http://images.amazon.com/images/P/0971880107.0...  
1  http://images.amazon.com/images/P/0316666343.0...  
2  http://images.amazon.com/images/P/0385504209.0...  
3  http://images.amazon.com/images/P/0060928336.0...  
4  http://images.amazon.com/images/P/0312195516.0...

In [ ]:
#Average Rating and Rating Count
average_rating = pd.DataFrame(ratings.groupby('ISBN')['bookRating'].mean())
average_rating['ratingCount'] = pd.DataFrame(ratings.groupby('ISBN')['bookRating'].count())
average_rating.sort_values('ratingCount', ascending=False).head()

In [14]:
#Recommendations based on Correlations
#To ensure statistical significance, users with less than 200 ratings and books with less than 100 ratings are excluded
counts1 = ratings['userID'].value_counts()
ratings = ratings[ratings['userID'].isin(counts1[counts1 >= 200].index)]
counts = ratings['bookRating'].value_counts()
ratings = ratings[ratings['bookRating'].isin(counts[counts >= 100].index)]

In [15]:
#Rating Matrix
#Convert to a 2D matrix - this table is blank, will be filled in after
ratings_pivot = ratings.pivot(index='userID',columns='ISBN').bookRating
userID = ratings_pivot.index
ISBN = ratings_pivot.columns
print(ratings_pivot.shape)
ratings_pivot.head()

(905, 207699)


ISBN     0330299891   0375404120   0586045007   9022906116   9032803328  \
userID                                                                    
254             NaN          NaN          NaN          NaN          NaN   
2276            NaN          NaN          NaN          NaN          NaN   
2766            NaN          NaN          NaN          NaN          NaN   
2977            NaN          NaN          NaN          NaN          NaN   
3363            NaN          NaN          NaN          NaN          NaN   

ISBN     9044922564   9044922572   9044922718   9044923161   904492401X  \
userID                                                                    
254             NaN          NaN          NaN          NaN          NaN   
2276            NaN          NaN          NaN          NaN          NaN   
2766            NaN          NaN          NaN          NaN          NaN   
2977            NaN          NaN          NaN          NaN          NaN   
3363            NaN          NaN          NaN          NaN          NaN   

ISBN      ...      UNGRANDHOMMED  X000000000  YOUTELLEM,AND  ZR903CX0003  \
userID    ...                                                              
254       ...                NaN         NaN            NaN          NaN   
2276      ...                NaN         NaN            NaN          NaN   
2766      ...                NaN         NaN            NaN          NaN   
2977      ...                NaN         NaN            NaN          NaN   
3363      ...                NaN         NaN            NaN          NaN   

ISBN    \0432534220\""  \2842053052\""  b00005wz75  cn108465  cn113107  \
userID                                                                   
254                NaN             NaN         NaN       NaN       NaN   
2276               NaN             NaN         NaN       NaN       NaN   
2766               NaN             NaN         NaN       NaN       NaN   
2977               NaN             NaN         NaN       NaN       NaN   
3363               NaN             NaN         NaN       NaN       NaN   

ISBN    Ô½crosoft  
userID             
254           NaN  
2276          NaN  
2766          NaN  
2977          NaN  
3363          NaN  

[5 rows x 207699 columns]

In [16]:
#Let's recommend books for people who like "The Lovely Bones: A Novel"
bones_ratings1 = ratings_pivot[('0316666343')]
similar_to_bones1 = ratings_pivot.corrwith(bones_ratings1)
corr_bones1 = pd.DataFrame(similar_to_bones1, columns = ['pearsonR'])
corr_bones1.dropna(inplace=True)
corr_summary1 = corr_bones1.join(average_rating['ratingCount'])
top10n = corr_summary1[corr_summary1['ratingCount']>=300].sort_values('pearsonR', ascending=False).head(10)
top10n

C:\Users\iknow\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3175: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\iknow\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3109: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


NameError: name 'average_rating' is not defined

In [ ]:
books_corr_to_bones1 = pd.DataFrame(top10n)
corr_books1 = pd.merge(books_corr_to_bones1, books, on='ISBN')
corr_books1 = corr_books1.drop(corr_books1.index[[0]])
print("If you likes The Lovely Bones we reommend the following: \n")
print(corr_books1.filter(['bookTitle','bookAuthor']))